# Analyse du dataset de Plonger 

## Importation des packadges

In [1]:
import numpy as np
import pandas as pd

## Resumé du dataset

Lecture du csv

In [2]:
dataset = pd.read_csv('AIDA_Results_IA_Institut.csv')

Affichage des 10 premières lignes

In [3]:
dataset.head(10)

,Start,Diver,Gender,Discipline,Line,Official Top,AP,RP,Card,Points,Remarks,Title Event,Event Type,Day,Category Event
0,1,Tasos Grillakis (GRC),M,FIM,NaN,00:00,33,23 m,YELLOW,12.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
1,2,Antonis Papantonatos (GRC),M,FIM,NaN,00:00,55,47 m,YELLOW,38.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
2,3,Dimitris Koumoulos (GRC),M,CNF,NaN,00:00,55,55 m,WHITE,55.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
3,4,Christos Papadopoulos (GRC),M,CWT,NaN,00:00,55,55 m,WHITE,55.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other
4,5,Anna Chalari (GRC),F,CWT,NaN,00:00,15,15 m,WHITE,15.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other
5,6,Alkisti Vouraki (GRC),F,CWT,NaN,00:00,16,16 m,WHITE,16.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other
6,7,Stavroula Kontorousi (GRC),F,CWT,NaN,00:00,20,20 m,WHITE,20.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other
7,8,Yuliya Ilkevich (BLR),F,CWT,NaN,00:00,40,31 m,YELLOW,21.0,"SHORT, TAG",Depth Event 2016,Depth Competition,2016-07-17,other
8,9,Dimitris Tsigkounis (GRC),M,CWT,NaN,00:00,40,26 m,YELLOW,11.0,"SHORT, TAG",Depth Event 2016,Depth Competition,2016-07-17,other
9,10,Emmanouil Marneros (GRC),M,CWT,NaN,00:00,47,47 m,WHITE,47.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other


In [4]:
dataset.shape

(26841, 15)

In [5]:
dataset.isna().sum()

Start                 0
Diver                 0
Gender                0
Discipline            0
Line              22147
Official Top          0
AP                    0
RP                    0
Card                  0
Points                0
Remarks               5
Title Event           0
Event Type            0
Day                   0
Category Event        0
dtype: int64

In [8]:
non_numeric_df = dataset['Diver','Gender', 'Discipline', 'Card', 'Remarks', 'Title Event', 'Event Type', 'Day', 'Category Event']
numeric_df = dataset['Official Top', 'AP', 'RP', 'Points']

KeyError: ('Diver', 'Gender', 'Discipline')